In [7]:
import spacy
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pickle as pkl

### NER extraction

In [2]:
nlp = spacy.load("en_core_web_sm")

In [11]:
doc = nlp(u"?")

In [37]:
def extract_filter_ents(text):
    doc = nlp(text)
    ents_extrd = []
    for ent in doc.ents:
        print(ent.label_)
        if ent.label_ in ["PERSON", "ORG", "NORP", "FAC", "GPE", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART"]:
            ents_extrd.append(ent.text)       
    return ents_extrd
        

In [51]:
extract_filter_ents("Winged Victory of Samothrace by Boticelli")

ORG


['Winged Victory of Samothrace']

### Nearest neighbours of the vector

In [33]:
def return_nn(matrix, vector, nn):
    nbrs = NearestNeighbors(nn)
    nbrs.fit(matrix)
    indices = nbrs.kneighbors(vector, return_distance=False)
    indices = indices[0]
#     print(matrix[tree.query(list(vector))[2]])
    
    

In [34]:
x = np.random.rand(1000, 200)
y = np.random.rand(1, 200)
print(y.shape)
return_nn(x, y, 10)

(1, 200)
[387 364 314 679 752 573  12 450 774 829]


### Zero shot model

In [75]:
class ZS_net(nn.Module):
        def __init__(self, question_emb_size):
            super(ZS_net, self).__init__()
            self.W = nn.Parameter(torch.nn.init.xavier_uniform_(torch.Tensor(question_emb_size, question_emb_size)).double())
            
        
        def forward(self, question_emb, label_emb):
            mult = torch.mm(question_emb, self.W)
            output = torch.bmm(mult.unsqueeze(1), label_emb.unsqueeze(2))
            return output


### Data loading

In [9]:
question_embedding = pkl.load(open('data/sentence_embeddings.pkl', 'rb'))
label_embedding = pkl.load(open('data/labels_embeddings.pkl', 'rb'))


In [10]:
targets = pkl.load(open('data/target.pkl', 'rb'))

In [16]:
label_embedding.shape

(103842, 4096)

In [76]:
model = ZS_net(4096)

In [51]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

### Shuffling of the data

In [28]:
indices = list(range(question_embedding.shape[0]))

In [29]:
np.random.shuffle(indices)

In [97]:
question_embedding_shuffled = np.zeros((103842, 4096))
label_embedding_shuffled = np.zeros((103842, 4096))
targets_shuffled = np.zeros((103842,))
q=0
for index in indices:
    print(q)
    print(index)
    print(question_embedding[index, :])
    question_embedding_shuffled[q,:] = question_embedding[index, :]
    label_embedding_shuffled[q,:] = label_embedding[index, :]
    targets_shuffled[q] = targets[index]
    q+=1

0
95775
[0. 0. 0. ... 0. 0. 0.]
1
15691
[ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
 -0.0328564 ]
2
79686
[0. 0. 0. ... 0. 0. 0.]
3
35070
[0. 0. 0. ... 0. 0. 0.]
4
12357
[ 0.00746889 -0.04898725  0.10116959 ...  0.00439976 -0.00057326
 -0.01259525]
5
71012
[0. 0. 0. ... 0. 0. 0.]
6
24207
[ 0.00746889 -0.04898725  0.05889944 ... -0.02379303 -0.04955649
 -0.00428714]
7
103837
[0. 0. 0. ... 0. 0. 0.]
8
87365
[0. 0. 0. ... 0. 0. 0.]
9
56130
[0. 0. 0. ... 0. 0. 0.]
10
56426
[0. 0. 0. ... 0. 0. 0.]
11
50857
[0. 0. 0. ... 0. 0. 0.]
12
101737
[0. 0. 0. ... 0. 0. 0.]
13
74511
[0. 0. 0. ... 0. 0. 0.]
14
52681
[0. 0. 0. ... 0. 0. 0.]
15
82281
[0. 0. 0. ... 0. 0. 0.]
16
95605
[0. 0. 0. ... 0. 0. 0.]
17
4117
[ 0.00746889 -0.02809111  0.05055163 ...  0.01024831  0.03176994
  0.03131968]
18
50493
[0. 0. 0. ... 0. 0. 0.]
19
30413
[0. 0. 0. ... 0. 0. 0.]
20
535
[ 0.00746889 -0.02809111  0.15019345 ...  0.01456291  0.00186402
 -0.02448341]
21
5572
[ 0.00746889 -0.04898725 -0.02151681 

 -0.03216166]
173
27256
[0. 0. 0. ... 0. 0. 0.]
174
83916
[0. 0. 0. ... 0. 0. 0.]
175
27728
[0. 0. 0. ... 0. 0. 0.]
176
13143
[ 0.00746889 -0.04600051  0.015286   ...  0.01150499  0.06855255
 -0.01259329]
177
33358
[0. 0. 0. ... 0. 0. 0.]
178
40681
[0. 0. 0. ... 0. 0. 0.]
179
71967
[0. 0. 0. ... 0. 0. 0.]
180
59885
[0. 0. 0. ... 0. 0. 0.]
181
92065
[0. 0. 0. ... 0. 0. 0.]
182
14708
[ 0.00746889 -0.05829022 -0.01133997 ...  0.07203656  0.06309941
 -0.01737966]
183
93869
[0. 0. 0. ... 0. 0. 0.]
184
60011
[0. 0. 0. ... 0. 0. 0.]
185
60928
[0. 0. 0. ... 0. 0. 0.]
186
53486
[0. 0. 0. ... 0. 0. 0.]
187
79430
[0. 0. 0. ... 0. 0. 0.]
188
10259
[ 0.00746889 -0.05823731  0.07881074 ... -0.00304284  0.03182752
 -0.0328564 ]
189
4336
[ 0.00746889 -0.04898725 -0.04877193 ...  0.02612866 -0.03562061
 -0.02695086]
190
35496
[0. 0. 0. ... 0. 0. 0.]
191
59508
[0. 0. 0. ... 0. 0. 0.]
192
46296
[0. 0. 0. ... 0. 0. 0.]
193
78384
[0. 0. 0. ... 0. 0. 0.]
194
28729
[0. 0. 0. ... 0. 0. 0.]
195
27651
[0. 0. 0.

56127
[0. 0. 0. ... 0. 0. 0.]
347
65178
[0. 0. 0. ... 0. 0. 0.]
348
86727
[0. 0. 0. ... 0. 0. 0.]
349
47945
[0. 0. 0. ... 0. 0. 0.]
350
60535
[0. 0. 0. ... 0. 0. 0.]
351
78560
[0. 0. 0. ... 0. 0. 0.]
352
30070
[0. 0. 0. ... 0. 0. 0.]
353
10276
[ 0.00746889 -0.04898725  0.02140611 ...  0.00859964 -0.01334015
 -0.03313944]
354
60554
[0. 0. 0. ... 0. 0. 0.]
355
61875
[0. 0. 0. ... 0. 0. 0.]
356
92076
[0. 0. 0. ... 0. 0. 0.]
357
67415
[0. 0. 0. ... 0. 0. 0.]
358
7018
[ 0.00746889 -0.02924411  0.0950251  ...  0.03500851  0.00556025
 -0.02310449]
359
97834
[0. 0. 0. ... 0. 0. 0.]
360
98302
[0. 0. 0. ... 0. 0. 0.]
361
34659
[0. 0. 0. ... 0. 0. 0.]
362
47707
[0. 0. 0. ... 0. 0. 0.]
363
13218
[0.00746889 0.03256541 0.02128621 ... 0.0433352  0.03189598 0.02286341]
364
67563
[0. 0. 0. ... 0. 0. 0.]
365
83846
[0. 0. 0. ... 0. 0. 0.]
366
80743
[0. 0. 0. ... 0. 0. 0.]
367
67742
[0. 0. 0. ... 0. 0. 0.]
368
31949
[0. 0. 0. ... 0. 0. 0.]
369
63423
[0. 0. 0. ... 0. 0. 0.]
370
82447
[0. 0. 0. ... 0. 0. 0

 -0.03591727]
509
23055
[ 0.00746889 -0.04332432  0.06820301 ...  0.0175476   0.0187634
 -0.01692696]
510
74071
[0. 0. 0. ... 0. 0. 0.]
511
88222
[0. 0. 0. ... 0. 0. 0.]
512
53320
[0. 0. 0. ... 0. 0. 0.]
513
77241
[0. 0. 0. ... 0. 0. 0.]
514
7214
[ 0.00746889 -0.05823731  0.06486309 ... -0.00328171  0.00408889
  0.0558305 ]
515
23650
[ 0.00746889  0.0115622   0.032343   ...  0.03874464  0.06012089
 -0.01687697]
516
18314
[ 0.00746889  0.17400269  0.03058697 ... -0.04885551 -0.00207114
 -0.02253188]
517
64389
[0. 0. 0. ... 0. 0. 0.]
518
28669
[0. 0. 0. ... 0. 0. 0.]
519
89034
[0. 0. 0. ... 0. 0. 0.]
520
7413
[ 0.00746889 -0.02924411 -0.01754067 ...  0.05244249  0.0895866
 -0.00738066]
521
35880
[0. 0. 0. ... 0. 0. 0.]
522
69574
[0. 0. 0. ... 0. 0. 0.]
523
20085
[ 0.00746889  0.08446027 -0.01279543 ... -0.02798249  0.0013367
 -0.04944398]
524
63858
[0. 0. 0. ... 0. 0. 0.]
525
6766
[ 0.00746889 -0.04898725  0.01631776 ... -0.00878661  0.0238608
  0.0130984 ]
526
47169
[0. 0. 0. ... 0. 0. 

87653
[0. 0. 0. ... 0. 0. 0.]
683
87187
[0. 0. 0. ... 0. 0. 0.]
684
41654
[0. 0. 0. ... 0. 0. 0.]
685
1180
[ 0.00746889 -0.04898725  0.13209489 ... -0.03558686 -0.0351264
 -0.00569315]
686
82973
[0. 0. 0. ... 0. 0. 0.]
687
80329
[0. 0. 0. ... 0. 0. 0.]
688
77227
[0. 0. 0. ... 0. 0. 0.]
689
2660
[ 0.00746889 -0.04898725  0.14783514 ...  0.01079598  0.09486738
 -0.00167731]
690
22776
[ 0.00746889 -0.02924411  0.01790771 ... -0.03177843  0.01924962
 -0.028478  ]
691
56953
[0. 0. 0. ... 0. 0. 0.]
692
52986
[0. 0. 0. ... 0. 0. 0.]
693
42817
[0. 0. 0. ... 0. 0. 0.]
694
86282
[0. 0. 0. ... 0. 0. 0.]
695
64939
[0. 0. 0. ... 0. 0. 0.]
696
86173
[0. 0. 0. ... 0. 0. 0.]
697
31156
[0. 0. 0. ... 0. 0. 0.]
698
97904
[0. 0. 0. ... 0. 0. 0.]
699
39984
[0. 0. 0. ... 0. 0. 0.]
700
63710
[0. 0. 0. ... 0. 0. 0.]
701
19074
[ 0.00746889  0.27713501  0.10623892 ... -0.05302583 -0.00047974
 -0.03317982]
702
36853
[0. 0. 0. ... 0. 0. 0.]
703
29808
[0. 0. 0. ... 0. 0. 0.]
704
66010
[0. 0. 0. ... 0. 0. 0.]
705
6

...

 -0.01385639]
1542
34286
[0. 0. 0. ... 0. 0. 0.]
1543
84395
[0. 0. 0. ... 0. 0. 0.]
1544
73143
[0. 0. 0. ... 0. 0. 0.]
1545
52079
[0. 0. 0. ... 0. 0. 0.]
1546
35448
[0. 0. 0. ... 0. 0. 0.]
1547
81707
[0. 0. 0. ... 0. 0. 0.]
1548
70739
[0. 0. 0. ... 0. 0. 0.]
1549
37008
[0. 0. 0. ... 0. 0. 0.]
1550
42290
[0. 0. 0. ... 0. 0. 0.]
1551
54032
[0. 0. 0. ... 0. 0. 0.]
1552
59937
[0. 0. 0. ... 0. 0. 0.]
1553
60571
[0. 0. 0. ... 0. 0. 0.]
1554
42804
[0. 0. 0. ... 0. 0. 0.]
1555
53419
[0. 0. 0. ... 0. 0. 0.]
1556
8326
[ 0.00746889 -0.02691392  0.06977036 ...  0.03551051  0.03381392
  0.01282423]
1557
51837
[0. 0. 0. ... 0. 0. 0.]
1558
75544
[0. 0. 0. ... 0. 0. 0.]
1559
37697
[0. 0. 0. ... 0. 0. 0.]
1560
7577
[ 0.00746889 -0.04898725 -0.01083057 ...  0.02608831  0.00563566
 -0.0143775 ]
1561
44672
[0. 0. 0. ... 0. 0. 0.]
1562
45308
[0. 0. 0. ... 0. 0. 0.]
1563
16575
[ 0.00746889  0.21385214  0.02733832 ... -0.02868306 -0.03416967
 -0.00363577]
1564
20545
[ 0.00746889 -0.05015421  0.07647987 ...  

 -0.01508914]
1736
50939
[0. 0. 0. ... 0. 0. 0.]
1737
80096
[0. 0. 0. ... 0. 0. 0.]
1738
45528
[0. 0. 0. ... 0. 0. 0.]
1739
38904
[0. 0. 0. ... 0. 0. 0.]
1740
45102
[0. 0. 0. ... 0. 0. 0.]
1741
8757
[ 0.00746889  0.00428057 -0.00628584 ...  0.02628727  0.0154434
 -0.0101474 ]
1742
9527
[ 0.00746889 -0.05109417  0.09705226 ...  0.02202149  0.07282314
 -0.03205393]
1743
27198
[0. 0. 0. ... 0. 0. 0.]
1744
49006
[0. 0. 0. ... 0. 0. 0.]
1745
58102
[0. 0. 0. ... 0. 0. 0.]
1746
90379
[0. 0. 0. ... 0. 0. 0.]
1747
63440
[0. 0. 0. ... 0. 0. 0.]
1748
8494
[ 0.00746889 -0.01741439 -0.04151007 ...  0.07119732  0.03052217
  0.00705283]
1749
79077
[0. 0. 0. ... 0. 0. 0.]
1750
28247
[0. 0. 0. ... 0. 0. 0.]
1751
25385
[ 0.00746889 -0.02809111  0.06985009 ...  0.01634206  0.02766423
 -0.01052537]
1752
48784
[0. 0. 0. ... 0. 0. 0.]
1753
13050
[ 0.00746889 -0.05291876 -0.02058594 ...  0.01247257  0.0159499
 -0.0179875 ]
1754
29880
[0. 0. 0. ... 0. 0. 0.]
1755
48544
[0. 0. 0. ... 0. 0. 0.]
1756
79328
[0. 0

[ 0.00746889  0.03408949  0.023245   ...  0.0443789   0.02560541
 -0.0271591 ]
2068
1156
[ 0.00746889 -0.09359213  0.03261573 ... -0.04038481  0.01789242
 -0.00777645]
2069
23267
[ 0.00746889 -0.04898725  0.04260714 ...  0.04015103 -0.00616745
 -0.05519399]
2070
37442
[0. 0. 0. ... 0. 0. 0.]
2071
101148
[0. 0. 0. ... 0. 0. 0.]
2072
90915
[0. 0. 0. ... 0. 0. 0.]
2073
103267
[0. 0. 0. ... 0. 0. 0.]
2074
41254
[0. 0. 0. ... 0. 0. 0.]
2075
47392
[0. 0. 0. ... 0. 0. 0.]
2076
82656
[0. 0. 0. ... 0. 0. 0.]
2077
29940
[0. 0. 0. ... 0. 0. 0.]
2078
81165
[0. 0. 0. ... 0. 0. 0.]
2079
99321
[0. 0. 0. ... 0. 0. 0.]
2080
62866
[0. 0. 0. ... 0. 0. 0.]
2081
66985
[0. 0. 0. ... 0. 0. 0.]
2082
17242
[ 0.00746889 -0.04898725  0.01500479 ...  0.06342773  0.01935594
 -0.01698719]
2083
40721
[0. 0. 0. ... 0. 0. 0.]
2084
70923
[0. 0. 0. ... 0. 0. 0.]
2085
50154
[0. 0. 0. ... 0. 0. 0.]
2086
62176
[0. 0. 0. ... 0. 0. 0.]
2087
7934
[ 0.00746889 -0.03087393  0.07886391 ...  0.00458214 -0.01886531
 -0.01504189]
2

 -0.06058411]
2284
10390
[ 0.00746889 -0.02626347  0.15870412 ...  0.05751736  0.02762693
 -0.03577995]
2285
14046
[ 0.00746889 -0.00251939  0.07142489 ...  0.03151967  0.01827157
 -0.01626734]
2286
65583
[0. 0. 0. ... 0. 0. 0.]
2287
23909
[ 0.00746889 -0.04898725 -0.01393307 ...  0.01776584  0.00563566
 -0.01703771]
2288
12673
[ 0.00746889 -0.05823731  0.16868776 ... -0.03474161 -0.00893057
 -0.02712752]
2289
30157
[0. 0. 0. ... 0. 0. 0.]
2290
42979
[0. 0. 0. ... 0. 0. 0.]
2291
14509
[ 0.00746889 -0.04898725  0.10869135 ...  0.02452984  0.01143883
  0.00873721]
2292
35819
[0. 0. 0. ... 0. 0. 0.]
2293
91545
[0. 0. 0. ... 0. 0. 0.]
2294
41630
[0. 0. 0. ... 0. 0. 0.]
2295
50686
[0. 0. 0. ... 0. 0. 0.]
2296
48760
[0. 0. 0. ... 0. 0. 0.]
2297
21643
[ 0.00746889  0.02645495  0.10405925 ...  0.03650516  0.00208425
 -0.01696755]
2298
37826
[0. 0. 0. ... 0. 0. 0.]
2299
64937
[0. 0. 0. ... 0. 0. 0.]
2300
102130
[0. 0. 0. ... 0. 0. 0.]
2301
90175
[0. 0. 0. ... 0. 0. 0.]
2302
59874
[0. 0. 0. ... 

KeyboardInterrupt: 

### Division into training/validation sets

In [85]:
num_epochs = 100
batch_size = 3
train_questions = question_embedding_shuffled[:int(0.7*question_embedding_shuffled.shape[0]), :]
train_label = label_embedding_shuffled[:int(0.7*label_embedding_shuffled.shape[0]), :]
train_y = targets_shuffled[:int(0.7*label_embedding_shuffled.shape[0])]
dev_questions = question_embedding_shuffled[int(0.7*question_embedding_shuffled.shape[0]):, :]
dev_label = label_embedding_shuffled[int(0.7*label_embedding_shuffled.shape[0]):, :]
dev_y = targets_shuffled[int(0.7*label_embedding_shuffled.shape[0]):]

In [86]:
dev_label.shape

(31153, 4096)

### training loop

In [90]:
for i in range(num_epochs):
    model.train()
#     for j in range(int(train_questions.shape[0] / batch_size)):
    for j in range(1):
        optimizer.zero_grad()
        curr_batch_questions = train_questions[j*batch_size:j*batch_size+batch_size, :]
        print(curr_batch_questions)
        curr_batch_label = train_label[j*batch_size:j*batch_size+batch_size, :]
        curr_batch_y = train_y[j*batch_size:j*batch_size+batch_size] 
#         print(curr_batch_label.shape)
        
        output = model(torch.from_numpy(curr_batch_questions).double(), torch.from_numpy(curr_batch_label).double())
        print(output.shape)
        loss = criterion(output.squeeze(1).squeeze(1), torch.from_numpy(curr_batch_y).double())
        print(loss)
        loss.backward()
        optimizer.step()

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
  -0.0328564 ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
torch.Size([3, 1, 1])
tensor(0.6992, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
  -0.0328564 ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
torch.Size([3, 1, 1])
tensor(0.6992, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
  -0.0328564 ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
torch.Size([3, 1, 1])
tensor(0.6992, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWith

   0.        ]]
torch.Size([3, 1, 1])
tensor(0.6992, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
  -0.0328564 ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
torch.Size([3, 1, 1])
tensor(0.6992, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
  -0.0328564 ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
torch.Size([3, 1, 1])
tensor(0.6992, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
  -0.0328564 ]
 [ 0.          0.          0.         ...

       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
  -0.0328564 ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
torch.Size([3, 1, 1])
tensor(0.6992, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
  -0.0328564 ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
torch.Size([3, 1, 1])
tensor(0.6992, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
  -0.0328564 ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
torch.Size([3, 1, 1])
tensor(0.6992, dtyp

   0.        ]]
torch.Size([3, 1, 1])
tensor(0.6992, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
  -0.0328564 ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
torch.Size([3, 1, 1])
tensor(0.6992, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
  -0.0328564 ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
torch.Size([3, 1, 1])
tensor(0.6992, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
  -0.0328564 ]
 [ 0.          0.          0.         ...

       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
  -0.0328564 ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
torch.Size([3, 1, 1])
tensor(0.6992, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.00746889 -0.04898725  0.00377392 ...  0.0308828   0.003251
  -0.0328564 ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
torch.Size([3, 1, 1])
tensor(0.6992, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


In [55]:
print(train_y[:100])

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]


In [93]:
question_embedding[:100]

array([[ 0.00746889, -0.01992372,  0.15822995, ...,  0.04556087,
         0.03342501, -0.00853212],
       [ 0.00746889, -0.02809111,  0.05006744, ...,  0.0178544 ,
        -0.01630899, -0.00498571],
       [ 0.00746889, -0.02809111,  0.01471707, ..., -0.00465224,
         0.01817099, -0.0229607 ],
       ...,
       [ 0.00746889,  0.01473884, -0.03301127, ...,  0.06051136,
        -0.01668801, -0.02728898],
       [ 0.00746889, -0.01522899,  0.05297965, ...,  0.0834971 ,
         0.03965647, -0.02430112],
       [ 0.00746889,  0.03841147,  0.08253884, ..., -0.01707941,
         0.03989314, -0.02083898]])

In [121]:
question_embedding[26000:26030]

array([[ 0.00746889, -0.04675408,  0.01665896, ...,  0.03874423,
         0.02462445, -0.01594856],
       [ 0.00746889, -0.05013892,  0.22888206, ..., -0.02310409,
         0.00337291, -0.03381677],
       [ 0.00746889, -0.04898725,  0.13325655, ..., -0.0125338 ,
        -0.03487156, -0.00794946],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [128]:
for i in range(50000,80000):
    print(targets[i])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


KeyboardInterrupt: 